# 📊 Unión y Limpieza de datos del Dataset  

---

**Objetivo del Notebook**  
Limpieza de datos, columnas innecesarias y valores nulos/blancos 

**Contexto del análisis**  
- Dataset de muestra proporcionado + csv proporcionado unido en un único excel dataset
- Enfoque en aprendizaje, validación del pipeline y comprensión del proceso

**Valor devuelto**  
- Copia del Dataset de muestra proporcionado completamente limpio y útil 

---




In [1]:
import pandas as pd

CSV_PATH = r"..\datos\OpportunityFieldHistory-16_1_2026.csv"
EXCEL_PATH = r"..\datos\DataSET_SF - copia.xlsx"
SHEET_BASE_NAME = "Oportunidades_CSV"
MAX_ROWS_EXCEL = 1_048_575  # un poco menos que el límite de Excel

with pd.ExcelWriter(EXCEL_PATH, engine="openpyxl", mode="a") as writer:
    chunk_iter = pd.read_csv(
        CSV_PATH, 
        sep=",", 
        encoding="utf-8", 
        quotechar='"', 
        chunksize=MAX_ROWS_EXCEL
    )

    for i, chunk in enumerate(chunk_iter):
        sheet_name = f"{SHEET_BASE_NAME}_{i+1}"
        chunk.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Hoja escrita: {sheet_name} ({len(chunk)} filas)")


FileNotFoundError: [Errno 2] No such file or directory: '..\\datos\\DataSET_SF - copia.xlsx'

In [ ]:


# ===============================
# 0. LEER ARCHIVO Y HOJAS
# ===============================
xls = pd.ExcelFile(EXCEL_PATH)
nombres_hojas = xls.sheet_names

# =================================
# 1. ELIMINAR COLUMNAS INNECESARIAS
# =================================
columnas_a_eliminar = [
    "Columna1", "Columna2", "..."
]

# ===============================
# 2. LIMPIEZA DE DATOS
# ===============================
hojas_limpias = {}
for nombre_hoja in nombres_hojas:
    df = pd.read_excel(xls, sheet_name=nombre_hoja)

    df.drop(columns=[c for c in columnas_a_eliminar if c in df.columns]) # eliminar columnas innecesarias
    df.dropna(how='all') # eliminar filas completamente vacías
    df.drop_duplicates(inplace=True) # eliminar duplicados exactos

    for col in df.select_dtypes(include=["float", "int"]).columns: # sustituir los valores NaN según corresponda
        df[col].fillna(0, inplace=True)
    for col in df.select_dtypes(include=["object"]).columns:
        df[col].fillna("Desconocido", inplace=True)
    for col in df.select_dtypes(include=["bool"]).columns:
        df[col].fillna(False, inplace=True)
    for col in df.select_dtypes(include=["datetime"]).columns:
        df[col].fillna(pd.Timestamp("1900-01-01"), inplace=True)
    
    #df_limpio = df.dropna(how='any')  # eliminar filas con NaN
    hojas_limpias[nombre_hoja] = df

# ===============================
# 3. GUARDAR EXCEL LIMPIO
# ===============================
with pd.ExcelWriter(r"..\datos\DataSET_SF_limpio.xlsx", engine='xlsxwriter') as writer:
    for nombre_hoja in nombres_hojas:
        hojas_limpias[nombre_hoja].to_excel(writer, sheet_name=nombre_hoja, index=False)

print("Archivo limpio guardado como 'DataSET_SF_limpio.xlsx'")




Archivo limpio guardado como 'DataSet Probabilidad Matrículas_limpio.xlsx'
                   ID           ACCOUNTID PL_CURSO_ACADEMICO  \
0  0061r00001KLSSJAA5  0011r00002YMDC6AAP          2022/2023   
1  0066900001W1pVKAAZ  0016900002mntULAAY          2022/2023   
2  0066900001cHyR5AAK  0016900002vnx7kAAA          2022/2023   
3  0066900001MevZwAAJ  0016900002aNZZuAAO          2022/2023   
4  0066900001VzuaIAAR  0016900002jiFCfAAM          2022/2023   

  PL_MES_ANIO_INICIO    PL_TIPO_ACCESO        RECORDTYPEID  \
0        Desconocido  Acceso a primero  012w0000000K4QPAA0   
1        Desconocido  Acceso a primero  012w0000000K4QPAA0   
2        Desconocido  Acceso a primero  012w0000000K4QTAA0   
3        Desconocido  Acceso a primero  012w0000000K4QPAA0   
4        Desconocido  Acceso a primero  012w0000000K4QTAA0   

              RECORDTYPENAME             PL_ESTADO            STAGENAME  \
0   Solicitud Admisión Grado              Validado        Reorientación   
1   Solicitud Adm

# 📝 Tratamiento tabla Oportunidades

---

**Objetivo del Notebook**  
Estadísticas de los pagos de los alumnos.

**Contexto del análisis**  
- Dataset de muestra proporcionado limpio
- Volumen reducido respecto al dataset real de producción  
- Enfoque en aprendizaje, validación del pipeline y comprensión del proceso

**Valor devuelto**  
- Estadísticas creadas

---

In [42]:
import utils
# ===================================================
# 4. LEER LA COLUMNA OPORTUNIDADES DEL ARCHIVO LIMPIO
# ===================================================
df_pagos = hojas_limpias["Oportunidad"]

# Convertir fechas relevantes a datetime
fechas_cols = [
    "MINIMUMPAYMENTDATE",
    "FECHALIMITEPRIMERPAGO",
    "DT_FECHA_CREACION_SA",
    "DT_FECHA_FIN_MATRICULA_OOGG"
]
for col in fechas_cols:
    if col in df_pagos.columns:
        df_pagos[col] = pd.to_datetime(df_pagos[col], errors="coerce")

# Importe total promedio, máximo y mínimo
importe_total_promedio = df_pagos["CU_IMPORTE_TOTAL"].mean()
importe_total_max = df_pagos["CU_IMPORTE_TOTAL"].max()
importe_total_min = df_pagos["CU_IMPORTE_TOTAL"].min()

# Porcentaje promedio pagado
porcentaje_pagado_promedio = df_pagos["PAID_PERCENT"].mean()

# Número de pagos superiores al mínimo
pagos_superiores = df_pagos[df_pagos["CH_PAGO_SUPERIOR"] == True].shape[0]

# Cumplimiento de plazos
df_pagos["cumple_plazo"] = pd.to_datetime(df_pagos["MINIMUMPAYMENTDATE"]) <= pd.to_datetime(df_pagos["FECHALIMITEPRIMERPAGO"])
porcentaje_cumplen_plazo = df_pagos["cumple_plazo"].mean() * 100

# Total descuentos aplicados y porcentaje promedio
total_descuentos = df_pagos["NU_IMPORTE_DESCUENTO_PRONTO_PA"].sum()
porcentaje_descuento_promedio = df_pagos["NU_PORCENTAJE_DESCUENTO_PRONTO"].mean()

# Deuda promedio
deuda_promedio = (df_pagos["CU_IMPORTE_TOTAL"] - df_pagos["PAID_AMOUNT"]).mean()

# Ratio de pago completo y morosidad
ratio_pago_completo = (df_pagos["PAID_PERCENT"] == 100).mean() * 100
ratio_morosidad = ((df_pagos["PAID_PERCENT"] < 100) & (pd.to_datetime(df_pagos["FECHALIMITEPRIMERPAGO"]) < pd.Timestamp.today())).mean() * 100

# Filtrar los que tienen porcentaje total pagado igual a 100
pagos_completos = df_pagos[df_pagos["PAID_PERCENT"] == 100].shape[0]

pagos_totales = df_pagos["PAID_AMOUNT"].mean()

# Seguimiento del pago mínimo
porcentaje_pago_minimo = (df_pagos["MINIMUMPAYMENTPAYED"] == True).mean() * 100
porcentaje_pago_superior = (df_pagos["CH_PAGO_SUPERIOR"] == True).mean() * 100
importe_promedio_pago_minimo = df_pagos["IMPORTEPRIMERPAGO"].mean()
tiempo_resolucion_a_pago_minimo = (df_pagos["MINIMUMPAYMENTDATE"] - df_pagos["DT_FECHA_CREACION_SA"]).mean()

# Bonificación por pronto pago
porcentaje_con_descuento = df_pagos["LK_DESCUENTO_PRONTO_PAGO"].notna().mean() * 100
importe_total_bonificado = df_pagos["NU_IMPORTE_DESCUENTO_PRONTO_PA"].sum()
ahorro_promedio_bonificado = df_pagos["NU_IMPORTE_DESCUENTO_PRONTO_PA"].mean()

# Relación matrícula y pagos
matriculados_pago_cero = ((df_pagos["PAID_AMOUNT"] == 0) & df_pagos["ESTADO_MATRICULA"].notna()).mean() * 100
matriculados_sin_pago_minimo = ((df_pagos["MINIMUMPAYMENTPAYED"] == False) & df_pagos["ESTADO_MATRICULA"].notna()).mean() * 100
tiempo_pago_minimo_a_matricula = (df_pagos["DT_FECHA_FIN_MATRICULA_OOGG"] - df_pagos["MINIMUMPAYMENTDATE"]).mean()

# Indicadores de flujo
tiempo_prueba_a_pago_minimo = (df_pagos["MINIMUMPAYMENTDATE"] - df_pagos["DT_FECHA_CREACION_SA"]).mean()
tiempo_pago_minimo_a_pago_total = ((df_pagos["PAID_AMOUNT"] == df_pagos["CU_IMPORTE_TOTAL"]) & df_pagos["MINIMUMPAYMENTDATE"].notna()).mean() * 100
pago_un_solo_plazo = ((df_pagos["PAID_PERCENT"] == 100) & (df_pagos["CH_PAGO_SUPERIOR"] == False)).mean() * 100


# =========================
# MOSTRAR RESULTADOS
# =========================
#print(utils.analisis_na_por_columna(df_pagos))
print("Número de alumnos que han pagado el 100%:", pagos_completos)
print("Media de pago:", pagos_totales.round(2))
print("Importe total promedio:", importe_total_promedio.round(2))
print("Importe total máximo:", importe_total_max.round(2))
print("Importe total mínimo:", importe_total_min.round(2))
print("Porcentaje promedio pagado:", porcentaje_pagado_promedio.round(2))
print("Pagos superiores al mínimo:", pagos_superiores)
print("Porcentaje que cumplen plazo:", porcentaje_cumplen_plazo.round(2))
print("Total descuentos aplicados:", total_descuentos.round(2))
print("Porcentaje promedio de descuento:", porcentaje_descuento_promedio.round(2))
print("Deuda promedio:", deuda_promedio.round(2))
print("Ratio pago completo (%):", ratio_pago_completo.round(2))
print("Ratio morosidad (%):", ratio_morosidad.round(2))

print("Porcentaje que han hecho el pago mínimo:", round(porcentaje_pago_minimo, 2))
print("Porcentaje que han pagado más del mínimo:", round(porcentaje_pago_superior, 2))
print("Importe promedio del pago mínimo:", round(importe_promedio_pago_minimo, 2))
print("Tiempo medio desde resolución a pago mínimo:", tiempo_resolucion_a_pago_minimo)

print("Porcentaje con descuento pronto pago:", round(porcentaje_con_descuento, 2))
print("Importe total bonificado:", round(importe_total_bonificado, 2))
print("Ahorro promedio por alumno bonificado:", round(ahorro_promedio_bonificado, 2))

print("Matriculados con pago total = 0 (%):", round(matriculados_pago_cero, 2))
print("Matriculados sin pago mínimo (%):", round(matriculados_sin_pago_minimo, 2))
print("Tiempo medio pago mínimo a matrícula:", tiempo_pago_minimo_a_matricula)

print("Tiempo medio prueba a pago mínimo:", tiempo_prueba_a_pago_minimo)
print("Porcentaje que pagan todo antes de matrícula:", round(tiempo_pago_minimo_a_pago_total, 2))
print("Porcentaje que pagan todo en un solo plazo:", round(pago_un_solo_plazo, 2))

Número de alumnos que han pagado el 100%: 5772
Media de pago: 2823.28
Importe total promedio: 3986.61
Importe total máximo: 39245.0
Importe total mínimo: -10350.0
Porcentaje promedio pagado: 23.5
Pagos superiores al mínimo: 9442
Porcentaje que cumplen plazo: 97.27
Total descuentos aplicados: 1319934.74
Porcentaje promedio de descuento: 0.18
Deuda promedio: 1163.33
Ratio pago completo (%): 12.18
Ratio morosidad (%): 83.06
Porcentaje que han hecho el pago mínimo: 0.0
Porcentaje que han pagado más del mínimo: 19.93
Importe promedio del pago mínimo: 716.38
Tiempo medio desde resolución a pago mínimo: -41765 days +09:27:36.188663296
Porcentaje con descuento pronto pago: 100.0
Importe total bonificado: 1319934.74
Ahorro promedio por alumno bonificado: 27.85
Matriculados con pago total = 0 (%): 72.7
Matriculados sin pago mínimo (%): 93.94
Tiempo medio pago mínimo a matrícula: 930 days 17:18:46.506647536
Tiempo medio prueba a pago mínimo: -41765 days +09:27:36.188663296
Porcentaje que pagan to

In [58]:
import pandas as pd

# =========================
# DATAFRAMES DE ORIGEN
# =========================
df_oportunidad = hojas_limpias["Oportunidad"]  # contiene PAID_AMOUNT
df_cuenta = hojas_limpias["Cuenta"]           # contiene CH_HIJO_EMPLEADO__PC y resto de datos

# =========================
# COLUMNAS CLAVE
# =========================
COL_ID_O = "ACCOUNTID"        # columna en df_oportunidad
COL_ID_C = "ID18__PC"                # columna en df_cuenta
COL_HIJO = "CH_HIJO_EMPLEADO__PC"
COL_PAGO = "PAID_AMOUNT"

# =========================
# NORMALIZACIONES
# =========================

# Hijo de empleado → booleano
df_cuenta[COL_HIJO] = (
    df_cuenta[COL_HIJO]
    .astype(str)
    .str.strip()
    .str.lower()
    .map({
        "true": True,
        "1": True,
        "yes": True,
        "false": False,
        "0": False,
        "no": False
    })
    .fillna(False)
)

# Pago → numérico
df_oportunidad[COL_PAGO] = pd.to_numeric(
    df_oportunidad[COL_PAGO],
    errors="coerce"
).fillna(0)

# =========================
# MERGE / JOIN
# =========================
df_merge = df_oportunidad.merge(
    df_cuenta[[COL_ID_C, COL_HIJO]],
    how="left",
    left_on=COL_ID_O,
    right_on=COL_ID_C
)

# =========================
# FILTRO: hijo de empleado + pago total = 0
# =========================
hijos_empleado_pago_cero = df_merge[
    (df_merge[COL_HIJO]) &
    (df_merge[COL_PAGO] == 0)
]

# =========================
# RESULTADOS
# =========================
print("Número de casos:", len(hijos_empleado_pago_cero))
porcentaje = len(hijos_empleado_pago_cero) / len(df_merge) * 100
print("Porcentaje sobre total de oportunidades: {:.2f}%".format(porcentaje))


if len(hijos_empleado_pago_cero) > 0:
    print("Ejemplos de hijos de empleado con pago = 0:")
    print(hijos_empleado_pago_cero.head(10))  # muestra los 10 primeros
else:
    print("No se encontraron casos de hijo de empleado con pago = 0.")

Número de casos: 0
Porcentaje sobre total de oportunidades: 0.00%
No se encontraron casos de hijo de empleado con pago = 0.
